In [1]:
import pandas as pd
import numpy as np
from IPython.display import display
import matplotlib.pyplot as plt

from imblearn.over_sampling import SMOTE
import gensim

from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_curve, precision_score, recall_score
from sklearn.metrics import classification_report

from utilities import remove_empty_tweets
from sklearn.metrics import classification_report, f1_score


#text is already cleaned.
#assign cleaned data to these variables.
train_data_path = 'cleaned_data/cleaned_train_data_for_subtask1.csv'
test_data_path = 'cleaned_data/cleaned_test_data_for_subtask1.csv'
#read files.
train_data = pd.read_csv(train_data_path)
test_data = pd.read_csv(test_data_path)

print("Train set:"% train_data.columns, train_data.shape, len(train_data)) 
print("Test set:"% test_data.columns, test_data.shape, len(test_data)) 



C:\Users\Mena\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Train set: (20974, 7) 20974
Test set: (4997, 7) 4997


In [5]:
#remove empty tweets.
train_data = remove_empty_tweets(train_data, "#2_tweet_clean_V2")
test_data = remove_empty_tweets(test_data, "#2_tweet_clean_V2")

In [6]:
#prepare train and test data.
X_train = train_data['#2_tweet_clean_V1']
y_train = train_data['#classes_id']
X_test = test_data['#2_tweet_clean_V1']
y_test = test_data['#classes_id']


In [7]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_no = 50000
sql_len = 250
dim = 100

tokenizer = Tokenizer(num_words=max_no, lower=False)
tokenizer.fit_on_texts(X_train.values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

X_tr = tokenizer.texts_to_sequences(X_train.values)
X_tr = pad_sequences(X_tr, maxlen=sql_len)
print('Shape of data tensor:', X_tr.shape)

tokenizer = Tokenizer(num_words=max_no, lower=False)
tokenizer.fit_on_texts(X_test.values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

X_te = tokenizer.texts_to_sequences(X_test.values)
X_te = pad_sequences(X_te, maxlen=sql_len)
print('Shape of data tensor:', X_te.shape)

Found 49126 unique tokens.
Shape of data tensor: (20974, 250)
Found 17491 unique tokens.
Shape of data tensor: (4997, 250)


In [8]:
Y_tr = pd.get_dummies(y_train).values
print('Shape of label tensor:', Y_tr.shape)

Y_te = pd.get_dummies(y_test).values
print('Shape of label tensor:', Y_te.shape)

Shape of label tensor: (20974, 21)
Shape of label tensor: (4997, 21)


In [9]:
import keras
import tensorflow
from keras.layers import Embedding, SpatialDropout1D, LSTM, Dense
#import tensorsflow_addons as tfa

model = keras.Sequential()
model.add(Embedding(max_no, dim, input_shape=(sql_len,)))
model.add(SpatialDropout1D(0.1))
model.add(LSTM(100, dropout=0.1, recurrent_dropout=0.2))
model.add(Dense(21, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 5
batch_size = 64

history = model.fit(X_tr, Y_tr, epochs=epochs, batch_size=batch_size,validation_split=0.3)

Using TensorFlow backend.



Train on 14681 samples, validate on 6293 samples
Epoch 1/5
14681/14681 [==============================] - 294s 20ms/step - loss: 2.6260 - accuracy: 0.2131 - val_loss: 2.5161 - val_accuracy: 0.2565
Epoch 2/5
14681/14681 [==============================] - 339s 23ms/step - loss: 2.2375 - accuracy: 0.3529 - val_loss: 2.3299 - val_accuracy: 0.3370
Epoch 3/5
14681/14681 [==============================] - 329s 22ms/step - loss: 1.4969 - accuracy: 0.5765 - val_loss: 2.4343 - val_accuracy: 0.3412
Epoch 4/5
14681/14681 [==============================] - 302s 21ms/step - loss: 0.8132 - accuracy: 0.7749 - val_loss: 2.7829 - val_accuracy: 0.3170
Epoch 5/5
14681/14681 [==============================] - 294s 20ms/step - loss: 0.4128 - accuracy: 0.8954 - val_loss: 3.1188 - val_accuracy: 0.3210


In [11]:
y_pred= model.predict(X_te)
y_pred = np.argmax(y_pred, axis=1)
y_test = test_data['#classes_id']
Y_te = pd.get_dummies(y_test).values

Y_te = np.argmax(Y_te, axis=1)
print(f1_score(Y_te, y_pred, average='macro'))

0.0463202492773424
